# Prototyping

Creates a neural network which evaluates a time series and produce a set of predicted values for the time series
Predicted values may be used in a policy to make a trade. This policy may be modeled by simple multiple regression or a neural network.

## Data
Test data is taken as most recent to avoid lookahead bias. Train data is split into a validation and training set during fitting.


## TODO
- Convert feature percentages to stdev
- Adding VIX as a signal
- Adding High/Low as signals
- Multiple securities/ aggregate samples
- Policy network
- Regularization (l2)
- Dilated convolution

In [22]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import layers

dataset_path = keras.utils.get_file("SPY.csv", "http://localhost:8000/data/daily/SPY.csv")

column_names = ['Date','Open','High','Low','Close','Adj Close','Volume'] 
raw_dataset = pd.read_csv(dataset_path, 
#                           names=column_names, 
                          dtype={'Close': np.float64,'Open': np.float64,'High': np.float64,'Adj Close': np.float64, 'Volume': np.float64},
                          header=0,
                          na_values = "?", 
                          comment='\t',
                          sep=",",
                          skipinitialspace=True)


dataset = raw_dataset.copy()
dataset = dataset.sort_values(by=['Date'],ascending=False)

dataset_stats = dataset.describe()
dataset_stats = dataset_stats.transpose()
NUM_INPUT_NEURONS = 64
NUM_OUTPUT_NEURONS = 3
NUM_SAMPLES = len(dataset)
NUM_TEST_SAMPLES = int(.33 * NUM_SAMPLES)
#NUM_TEST_SAMPLES = 1000
print NUM_SAMPLES
# Create features (only close price for now)
def convert_to_percentage(old, new):
    return (old - new) / old


def convert_labels_to_category(labels): 
    # Simplification - If positive return, 1, else 0
    # return map(lambda arr: 1 if arr[0] > 1 else 0, labels)
    # rounding simpliciation, 10th of percentage
    return map(lambda arr: map(lambda val: round(val,2),arr), labels)

def convert_to_train(raw_dataset):
    dataset = raw_dataset.copy()
    features = []
    labels = []
    for i in range(5, len(dataset) - NUM_INPUT_NEURONS):

        feature_dataset = dataset[i:i+NUM_INPUT_NEURONS].copy()
        latest_close = feature_dataset['Close'].iloc[0]
        
        features.append(
            feature_dataset['Close']
                .map(lambda current: convert_to_percentage(latest_close, current))
                .tolist()
        )
        labels.append([
            dataset['Close'].iloc[i-1] / latest_close, # 1 day trade
            dataset['Close'].iloc[i-3] / latest_close, # 3 day trade
            dataset['Close'].iloc[i-5] / latest_close, # 5 day trade
        ])
        
    # Without converting labels the precision is hard to determine accuracy. 
    # Rather than crude 0/1, maybe this can be more sophisticated
    labels = convert_labels_to_category(labels)
    
    return [features,labels]
converted_feature_set = convert_to_train(dataset)
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(converted_feature_set[0][0])]),
    layers.Dense(32, activation=tf.nn.relu),
    layers.Dense(16, activation=tf.nn.relu),
    layers.Dense(NUM_OUTPUT_NEURONS)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.001)

  model.compile(loss='mse',
                optimizer='sgd',
                metrics=[
                    'mae',
                #    'accuracy'
                ])
  return model

tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

model = build_model()
model.summary()



6445
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_9 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_10 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 51        
Total params: 6,819
Trainable params: 6,819
Non-trainable params: 0
_________________________________________________________________


In [23]:
#len(converted_feature_set[0][0])
print NUM_TEST_SAMPLES
train_data = np.array(converted_feature_set[0][NUM_TEST_SAMPLES:])
train_labels = np.array(converted_feature_set[1][NUM_TEST_SAMPLES:])

test_data = np.array(converted_feature_set[0][:NUM_TEST_SAMPLES])
test_labels = np.array(converted_feature_set[1][:NUM_TEST_SAMPLES])


2126


In [24]:
print train_data
print train_labels

[[ 0.00000000e+00  7.90315238e-03  8.16085360e-03 ...  4.34671061e-02
   3.90000932e-02  4.55287589e-02]
 [ 0.00000000e+00  2.59754096e-04  3.46358998e-04 ...  3.13446624e-02
   3.79253362e-02  4.19949700e-02]
 [ 0.00000000e+00  8.66274042e-05  4.24387670e-03 ...  3.76753684e-02
   4.17460596e-02  4.71159020e-02]
 ...
 [ 0.00000000e+00  2.80308653e-03  6.30582347e-03 ... -8.40925960e-03
  -9.11115247e-03 -4.90652267e-03]
 [ 0.00000000e+00  3.51258301e-03  9.83838070e-03 ... -1.19477295e-02
  -7.73128065e-03  2.81096592e-03]
 [ 0.00000000e+00  6.34809591e-03  7.75853315e-03 ... -1.12834979e-02
  -7.04090268e-04  1.41043724e-03]]
[[1.01 1.   1.01]
 [1.01 1.01 1.01]
 [1.   1.01 1.  ]
 ...
 [1.   1.   1.  ]
 [1.   1.   1.  ]
 [1.   1.   1.  ]]


In [25]:

history = model.fit(
    train_data, train_labels,
    epochs=60, validation_split = 0.2, verbose=1,
    callbacks=[tensorboard]
)

Train on 3400 samples, validate on 850 samples
Epoch 1/60
3400/3400 [==============================] - 0s 107us/step - loss: 0.2823 - mean_absolute_error: 0.4536 - val_loss: 0.0088 - val_mean_absolute_error: 0.0865
Epoch 2/60
3400/3400 [==============================] - 0s 48us/step - loss: 0.0039 - mean_absolute_error: 0.0476 - val_loss: 0.0011 - val_mean_absolute_error: 0.0264
Epoch 3/60
3400/3400 [==============================] - 0s 50us/step - loss: 0.0026 - mean_absolute_error: 0.0381 - val_loss: 9.2978e-04 - val_mean_absolute_error: 0.0242
Epoch 4/60
3400/3400 [==============================] - 0s 47us/step - loss: 0.0023 - mean_absolute_error: 0.0355 - val_loss: 8.1872e-04 - val_mean_absolute_error: 0.0226
Epoch 5/60
3400/3400 [==============================] - 0s 42us/step - loss: 0.0021 - mean_absolute_error: 0.0335 - val_loss: 7.2897e-04 - val_mean_absolute_error: 0.0214
Epoch 6/60
3400/3400 [==============================] - 0s 36us/step - loss: 0.0020 - mean_absolute_error

Epoch 48/60
3400/3400 [==============================] - 0s 41us/step - loss: 7.7409e-04 - mean_absolute_error: 0.0195 - val_loss: 2.1088e-04 - val_mean_absolute_error: 0.0110
Epoch 49/60
3400/3400 [==============================] - 0s 47us/step - loss: 7.6913e-04 - mean_absolute_error: 0.0194 - val_loss: 2.0817e-04 - val_mean_absolute_error: 0.0109
Epoch 50/60
3400/3400 [==============================] - 0s 49us/step - loss: 7.6450e-04 - mean_absolute_error: 0.0194 - val_loss: 2.0772e-04 - val_mean_absolute_error: 0.0109
Epoch 51/60
3400/3400 [==============================] - 0s 37us/step - loss: 7.6042e-04 - mean_absolute_error: 0.0193 - val_loss: 2.0986e-04 - val_mean_absolute_error: 0.0110
Epoch 52/60
3400/3400 [==============================] - 0s 41us/step - loss: 7.5613e-04 - mean_absolute_error: 0.0193 - val_loss: 2.0455e-04 - val_mean_absolute_error: 0.0108
Epoch 53/60
3400/3400 [==============================] - 0s 42us/step - loss: 7.5171e-04 - mean_absolute_error: 0.0192 -

In [26]:
outputs = model.predict(test_data)


In [27]:

# one day return
actual = map(lambda arr: arr[0],outputs)
# signal step for our policy network
signals = map(lambda arr: sum(arr) / len(arr),outputs)
# primitive policy temporarily in place of a RL policy network
trades = map(lambda signal: 1 if round(signal,2) > 1 else 0,signals)

In [28]:
df = pd.DataFrame({
    'signal':signals,
#    'actual':test_labels,
    'actual': actual,
    'trade':trades,
})

In [29]:
df['entry_success'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 1) else 0,axis=1)
df['entry_failure'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 1) else 0,axis=1)
df['avoid_success'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 0) else 0,axis=1)
df['avoid_failure'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 0) else 0,axis=1)


In [30]:
# primitive policy - replace with a policy network which maximizes reward
def label_success (row):
    return 0 if (row['entry_failure'] == 1 or row['entry_failure'] == 1) else 1

success = df.apply (lambda row: label_success (row),axis=1)
df['success'] = success;
df

,actual,signal,trade,entry_success,entry_failure,avoid_success,avoid_failure,success
0,0.994335,0.991441,0,0,0,1,0,1
1,0.993333,0.993553,0,0,0,1,0,1
2,0.995984,0.994850,0,0,0,1,0,1
3,0.997563,0.997535,0,0,0,1,0,1
4,1.001283,0.995668,0,0,0,0,1,1
5,1.002498,0.996641,0,0,0,0,1,1
6,1.002811,0.997913,0,0,0,0,1,1
7,0.998282,0.995376,0,0,0,1,0,1
8,1.003007,1.001677,0,0,0,0,1,1
9,0.998767,1.000271,0,0,0,1,0,1


In [31]:
print '\nNon-loss events'
print sum(df['success'])
print sum(df['success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nLose trades'
print sum(df['entry_failure'])
print sum(df['entry_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nWin trades'
print sum(df['entry_success'])
print sum(df['entry_success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nMissed opportunities'
print sum(df['avoid_failure'])
print sum(df['avoid_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nBullets dodged'
print sum(df['avoid_success'])
print sum(df['avoid_success']) / (NUM_TEST_SAMPLES * 1.00)


Non-loss events
2086
0.981185324553

Lose trades
40
0.0188146754468

Win trades
460
0.216368767639

Missed opportunities
652
0.306679209784

Bullets dodged
974
0.458137347131
